In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("navoneel/brain-mri-images-for-brain-tumor-detection")

print("Path to dataset files:", path)

100%|██████████| 15.1M/15.1M [00:00<00:00, 89.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/navoneel/brain-mri-images-for-brain-tumor-detection/versions/1


In [ ]:
data_dir="/root/.cache/kagglehub/datasets/navoneel/brain-mri-images-for-brain-tumor-detection/versions/1/brain_tumor_dataset"

In [ ]:
transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48,48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [ ]:
dataset=ImageFolder(data_dir,transform=transform)

In [ ]:
len(dataset)

253

In [ ]:
classes=dataset.classes
classes

['no', 'yes']

In [ ]:
0.7*253

177.1

In [ ]:
253-177

76

In [ ]:
train_data,test_data=random_split(dataset,[177,76])

In [14]:
train_loader=DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader=DataLoader(test_data,batch_size=32,shuffle=False)

In [15]:
import torch.nn as nn
import torch.optim as optim

In [16]:
class MyNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.model=nn.Sequential(
        nn.Conv2d(1,32,3,padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2,2),

        nn.Conv2d(32,64,3,padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2,2),

        nn.Flatten(),
        nn.Linear(64*12*12,128),
        nn.ReLU(),
        nn.Linear(128,2)
    )

  def forward(self,x):
    return self.model(x)

In [20]:
model=MyNN()

In [21]:
loss_func=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [22]:
for epoch in range(20):
  running_loss=0.0
  for i,label in train_loader:
    out=model(i)
    loss=loss_func(out,label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  print(f"Epoch: {epoch+1}   Loss: {running_loss/len(train_loader)}")

Epoch: 1   Loss: 1.9523468216260274
Epoch: 2   Loss: 0.6666730344295502
Epoch: 3   Loss: 0.6191863516966502
Epoch: 4   Loss: 0.5740624715884527
Epoch: 5   Loss: 0.5297210911909739
Epoch: 6   Loss: 0.4928533335526784
Epoch: 7   Loss: 0.42352573573589325
Epoch: 8   Loss: 0.37562362353007
Epoch: 9   Loss: 0.37958522140979767
Epoch: 10   Loss: 0.32916442553202313
Epoch: 11   Loss: 0.33530621230602264
Epoch: 12   Loss: 0.37479570011297864
Epoch: 13   Loss: 0.29587238778670627
Epoch: 14   Loss: 0.23423967013756433
Epoch: 15   Loss: 0.17816070343057314
Epoch: 16   Loss: 0.1629988228281339
Epoch: 17   Loss: 0.11816524714231491
Epoch: 18   Loss: 0.1013018445422252
Epoch: 19   Loss: 0.07506011053919792
Epoch: 20   Loss: 0.07302279366801183


In [23]:
model.eval

<bound method Module.eval of MyNN(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=9216, out_features=128, bias=True)
    (8): ReLU()
    (9): Linear(in_features=128, out_features=2, bias=True)
  )
)>

In [24]:
total=0
correct=0
with torch.no_grad():
  for i,labels in test_loader:
    out=model(i)
    _,pred=torch.max(out,1)
    total+=labels.size(0)
    correct+=(pred==labels).sum().item()
print(f"Accuracy: {100*(correct/total)}")

Accuracy: 75.0


In [25]:
path="brain_tumor_mri_pytorch.pth"
torch.save(model.state_dict(),path)

In [26]:
total=0
correct=0
with torch.no_grad():
  for i,labels in train_loader:
    out=model(i)
    _,pred=torch.max(out,1)
    total+=labels.size(0)
    correct+=(pred==labels).sum().item()
print(f"Accuracy: {100*(correct/total)}")

Accuracy: 98.30508474576271
